In [1]:
import pandas as pd
import re

In [3]:
df = pd.read_csv('C:\\Users\\AZhar ALi\\Downloads\\Restaurant_Reviews (1).tsv', sep='\t')

In [4]:
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [24]:
df

,Review,Liked,processed_review
0,Wow... Loved this place.,1,wow love place
1,Crust is not good.,0,crust good
2,Not tasty and the texture was just nasty.,0,tasti textur nasti
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...
4,The selection on the menu was great and so wer...,1,select menu great price
...,...,...,...
995,I think food should have flavor and texture an...,0,think food flavor textur lack
996,Appetite instantly gone.,0,appetit instantli gone
997,Overall I was not impressed and would not go b...,0,overal impress would go back
998,"The whole experience was underwhelming, and I ...",0,whole experi underwhelm think go ninja sushi n...


In [5]:
df.shape

(1000, 2)

In [6]:
import nltk

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\AZhar
[nltk_data]     ALi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [8]:
list_stopwords = nltk.corpus.stopwords.words('english')

In [9]:
stemmer = nltk.stem.porter.PorterStemmer()
stemmer.stem('lovely')

'love'

In [10]:
def preprocess(review):
    review = review.lower()
    review = re.sub('[^a-z]',' ',review) # remove unwanted chars
    words = [stemmer.stem(word) for word in review.split() if not word in list_stopwords] # remove stopwords
    return ' '.join(words)

In [11]:
preprocess('Wow... Loved this place.')

'wow love place'

In [12]:
df['processed_review'] = df['Review'].apply(preprocess)

In [13]:
# To prepare BOW table

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
vectorizer = CountVectorizer(max_features=500) # most freq 500 words

In [16]:
vectorizer.fit(df['processed_review']) # unique words with freq....

CountVectorizer(max_features=500)

In [17]:
len(vectorizer.get_feature_names())

500

In [18]:
bow_table = vectorizer.transform(df['processed_review']) #Sparse Format(Compressed)
bow_table = bow_table.toarray()
bow_table.shape

(1000, 500)

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
model = RandomForestClassifier(random_state=100)
model.fit(bow_table, df['Liked'])

RandomForestClassifier(random_state=100)

In [21]:
model.score(bow_table,df['Liked']) #training score

0.987

In [22]:
# Predictions on real reviews => function
def get_sentiment(review):
    review = preprocess(review)
    numeric_array = vectorizer.transform([review])
    return model.predict(numeric_array)

In [23]:
get_sentiment('far away from city. bad service.')

array([0], dtype=int64)

In [48]:
get_sentiment('taste is very good')

array([1], dtype=int64)